# **Computer Vision 600.661: Homework 2**
## **3D Camera Modeling**

### Dimitri Lezcano

### **Imports**

In [1]:
%matplotlib inline
import numpy as np
import numpy.linalg as la
import cv2
import matplotlib.pyplot as plt

### *Previous Functions*

Affine Transformations

In [2]:
def affine_2d_Ab(X, Y):
    ''' create the A matrix for affine transformation
    
        Args:
            - X: the source points ( N x 2 )
            - Y: the target points ( N x 2 )
            
        Returns:
            - A: the A (2*N x 6) matrix for least squares formulation
            - b: the b (2*N) vector for least squares formulation
    
    '''
    # initialize the matrices
    A = np.zeros( ( 2*X.shape[0], 6 ) )
    b = Y.reshape(-1)
    
    Xh = np.hstack( ( X, np.ones( ( X.shape[0], 1 ) ) ) )
#     Yh = np.hstack( ( Y, np.ones( ( X.shape[0], 1 ) ) ) )

    A[0::2, :3] = Xh
    A[1::2, 3:] = Xh
    
    
    
    return A, b
    
# affine_2d_Ab

def get_affinetransform(X, Y):
    ''' wrapper function for ease of use for affine transform'''
    A, b = affine_2d_Ab(source, target)
    t = la.inv(A.T @ A) @ A.T @ b # transform vector
    affine_transform = np.vstack((t.reshape(2,3), [0, 0, 1]))
    
    return affine_transform

# get_affinetransform

Homographies

In [3]:
def homography_2d_A(X, Y):
    ''' create the A matrix for homography transformation
    
        Args:
            - X: the source points ( N x 2 )
            - Y: the target points ( N x 2 )
            
        Returns:
            - A: the A (2*N x 9) matrix for least squares formulation
    
    '''
    # initialize the matrices
    A = np.zeros( ( 2*X.shape[0], 9 ) )
    
    Xh = np.hstack( ( X, np.ones( ( X.shape[0], 1 ) ) ) )

    A[:, 0:6] = affine_2d_Ab(X, Y)[0] 
    
    A[0::2, 6:] = -Y[:,0].reshape(-1,1) * Xh
    A[1::2, 6:] = -Y[:,1].reshape(-1,1) * Xh
    
    return A
    
# homography_2d_A

def solveA_svd(A):
    ''' Function to return the solution of:
        
        argmin_x ||A x||^2 
        
        Args:
            - A: the matrix
            
        Return:
            - x = argmin of above 1-D vector
            - s = the associated singular value
    
    '''
    # perform the svd of A
    u, s, vh = la.svd(A)
    
    # last singular value is the smallest 
    s_min = s[-1]
    v_min = vh[-1,:]
    v_min = v_min/la.norm(v_min)
    
    return v_min, s_min
    
# solveA_svd

def get_homography(X, Y):
    ''' wrapper function for ease of use for homography'''
    A = homography_2d_A(X, Y)

    t, s = solveA_svd(A)
    T = t.reshape(3,3)
    
    return T

# get_homography

def apply_homography(T, X):
    ''' applies homography to row-wise coordinates
        
        Args:
            - T  ( 3 x 3 ) homography transform
            - X  ( N x 2 ) points to be transform
    
    '''
    Xh = np.hstack( (X, np.ones((X.shape[0], 1)) ) )
    
    Yh = Xh @ T.T # project coordinates
    
    Yh /= Yh[:,-1].reshape(-1, 1) # normalize the scale coordinate
    
    return Yh[:,0:2]

# apply_homography    

Gradient Histogramming

In [4]:
def gradient_histw(grads, n_bins = 10, plot:bool=False):
    ''' function to compute the weighted histogram of gradients
        
        Args:
            - grads: [N x 2] array of (x, y) gradient vectors
            - n_binds: integer of the number of bins to use (Default = 10)
            - plot: plotting argument
            
        Returns:
            - theta: a `n_bins`-vector of the directions of gradients
            - histw: a `n_bins`-vector of the weighted histogram
            
    
    '''
    # compute theta array
    theta = np.linspace(0, 2*np.pi, n_bins)
    
    # compute gradient directions and magnitudes
    theta_grads = np.arctan2(grads[:,1], grads[:,0])
    mag_grads = la.norm(grads, axis=1)
    
    # reorient so angles are between 0 and 2*pi
    theta_grads[theta_grads < 0] += 2*np.pi
    
    # compute a histogram
    histw, bins = np.histogram(theta_grads, bins=theta, weights=mag_grads)
    
    if plot:
        plt.hist(np.rad2deg(theta_grads), bins=np.rad2deg(theta), weights=mag_grads)
        plt.xlabel('theta (degs)')
        plt.title('Histogram of Gradients')
    
    
    return theta, histw
    
# gradient_histw

RANSAC

In [5]:
def get_keypoint_coord_from_match(matches, kp1, kp2, index):
    """ Gets the keypoint coordinates that correspond to matches[index].
      For example, if we want to get the coordinates of the keypoints corresponding
      to the 10th matching pair, we would be passing

              get_keypoint_coord_from_match(matches, kp1, kp2, 10)

      Then it will return keypoint1, keypoint2, where
      keypoint1: [x, y] coordinate of the keypoint in img1 that corresponds to matches[10]
      keypoint2: [x, y] coordinate of the keypoint in img2 that corresponds to matches[10]
    """
    keypoint1 = [kp1[matches[index].queryIdx].pt[0], kp1[matches[index].queryIdx].pt[1]]
    keypoint2 = [kp2[matches[index].trainIdx].pt[0], kp2[matches[index].trainIdx].pt[1]]
    return keypoint1, keypoint2

# get_keypoint_coord_from_match

def ransac(matches, kp1, kp2, num_samples: int = 6, num_trials: int = 3000,
           inlier_thresh: int = 5):
    
    # Some parameters
    total_matches = len(matches)
    
    # To keep track of the best transformation
    xform = np.zeros((3, 3))
    most_inliers = 0

    # turn the keypts into a numpy array: rows are the x-y coordinates
    keypts1 = []
    keypts2 = []
    for idx in range(total_matches):
        keypt1, keypt2 = get_keypoint_coord_from_match(matches, kp1, kp2, idx)
        
        keypts1.append(keypt1)
        keypts2.append(keypt2)
    # for
    
    keypts1 = np.array(keypts1).astype(np.float32)
    keypts2 = np.array(keypts2).astype(np.float32)
    
    # Loop through num_trials times
    for i in range(num_trials):

        # Randomly choose num_samples indices from total number of matches
        choices = np.random.choice(total_matches, num_samples, replace=False)

        # Get the matching keypoint coordinates from those indices
        keypts1_choice = keypts1[choices,:]
        keypts2_choice = keypts2[choices,:]

        # get homography   
        xform_i = get_homography(keypts1_choice, keypts2_choice)

        # count the number of inliers
        keypts1_xform_i = apply_homography(xform_i, keypts1)
        dists_i = la.norm(keypts1_xform_i - keypts2, axis=1)
        num_inliers = np.count_nonzero(dists_i <= inlier_thresh)

        # If for this transformation we have found the most inliers update most_inliers and xform
        if num_inliers > most_inliers:
            most_inliers = num_inliers
            xform = np.copy(xform_i)

        # if

    # for
    
    return xform
    
# ransac

Image Homography calculations using feature matching

In [6]:
def compute_img_homography(img1, img2, num_samples: int = 6, num_trials: int = 3000,
           inlier_thresh: int = 5):
    ''' Function to compute the image homography using RANSAC
        Computes transform to warp img1 -> img2
    
    '''
    
    # orb feature detector
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    # create a BFMather object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    # match descriptors
    matches = bf.match(des1, des2)
    
    # sort them in the order of their distance
    matches = sorted(matches, key = lambda x: x.distance)
    
    xform = ransac(matches, kp1, kp2, num_samples=num_samples,
                  num_trials=num_trials, inlier_thresh=inlier_thresh)
    
    return xform

# compute_img_homography

Inverse warping function: Previous Code

In [7]:
def get_pixel_contributions(x, y):
    """ Gets the overlapped pixels along with the relative contributions.
      Args:
        x: The x-coordinate
        y: The y-coordinate
      Return:
        pixel_contributions: List of [x, y, contribution] where (x, y) are the coordinates of
                             pixels that the input overlaps with.
    """

    pixel_contributions = []
    
    # find the ranges of x
    if int(x) == x and False: # integer
        range_x = [int(x)]
        
    else:
        range_x = [int(x), int(x) + 1]
        
    # find the ranges of y
    if int(y) == y and False: # integer
        range_y = [int(y)]
        
    else:
        range_y = [int(y), int(y) + 1]
        
    # iterate through the ranges
    for xi in range_x:
        for yi in range_y:
            # calculate the overlap
            overlap_x = 1-abs(x - xi)
            overlap_y = 1-abs(y - yi)
            
            # calculate the contribution
            contribute = overlap_x * overlap_y
            
            # add the contribution
            pixel_contributions.append([xi, yi, contribute])
            
        # for
    # for
    
    return np.array(pixel_contributions)

# get_pixel_contributions

def get_pixel_contributions_arr(X, Y):
    pixel_contributions = np.zeros((4*X.shape[0], 3))
    
    Xr = X.astype(int)
    Yr = Y.astype(int)
    
    Xrep = X.repeat(4)
    Yrep = Y.repeat(4)
    
    # set the x-coordinates
    pixel_contributions[0::4, 0] = Xr
    pixel_contributions[1::4, 0] = Xr
    pixel_contributions[2::4, 0] = Xr + 1
    pixel_contributions[3::4, 0] = Xr + 1
    
    # set the y-coordinates
    pixel_contributions[0::4, 1] = Yr
    pixel_contributions[2::4, 1] = Yr
    pixel_contributions[1::4, 1] = Yr + 1
    pixel_contributions[3::4, 1] = Yr + 1
    
    # measure the contributions
    overlap_x = 1 - np.abs(pixel_contributions[:,0] - Xrep)
    overlap_y = 1 - np.abs(pixel_contributions[:,1] - Yrep)
    
    pixel_contributions[:,2] = overlap_x * overlap_y
    
    return pixel_contributions
    
# get_pixel_contributions_arr

def inverse_warp(transform, image, cols:int= None, rows:int=None):
    
    # image checking
    if image.ndim < 3:
        image = np.expand_dims(image, 2).repeat(3, axis=2)
        
    # if
    
    elif image.shape[2] == 1:
        image = image.repeat(3, axis=2)
        
    # elif
    
    if cols is None:
        cols = image.shape[1]
    
    if rows is None:
        rows = image.shape[0]

    output_image = np.zeros((rows, cols, 3), dtype=float)

    # Invert the transformation matrix
    transform = la.inv(transform)
    
    # get the pixel coordinates
    X, Y = np.meshgrid(np.arange(cols), np.arange(rows))
    pts = np.hstack((X.reshape(-1,1), Y.reshape(-1,1)))
    
    # transform the pixel coordinates
    pts_transformed = apply_homography(transform, pts) 
    
    # get the pixel contributions shape rows*cols x 4 x 3
    contributions = get_pixel_contributions_arr(pts_transformed[:,0], pts_transformed[:,1])
    idxs = contributions[:,0:2].astype(int)
    
    invalid_idxs = (idxs < 0).any(axis=1) | (idxs[:,1] >= image.shape[0]) | (idxs[:,0] >= image.shape[1])
    
    # cancel out contributions and default invalid index to origin
    contributions[invalid_idxs, :] = 0
    idxs = contributions[:,0:2].astype(int) # reset the idxs
    
    # get the contributions
    contrib_pxs = contributions[:,-1].reshape(-1,1) * image[idxs[:,1], idxs[:,0]]
    
    # perform the inverse warping 
    output_image[pts[:,1], pts[:,0]] += contrib_pxs[0::4]
    output_image[pts[:,1], pts[:,0]] += contrib_pxs[1::4]
    output_image[pts[:,1], pts[:,0]] += contrib_pxs[2::4]
    output_image[pts[:,1], pts[:,0]] += contrib_pxs[3::4]
        

    return output_image.astype(np.uint8)

# inverse_warp

Image stitching functions

In [8]:
def blend_images(image1, image2):
    ''' Blends the images from image1 and image2
        We assume that the image1 warped into plane of image2
        
        Method: simple feather blending on overlaps
    '''
    # compute the alpha matrix
    a1 = np.any(image1 != 0, axis=2) * 1
    a2 = np.any(image2 != 0, axis=2) * 0.3
    
    alpha = np.expand_dims(a1 - a2, axis=2)
    alpha[alpha < 0] = 0
    
    # blend the images
    image = alpha * image1 + ( 1 - alpha) * image2
    
    return image.astype(int)

# blend_images

def pw_homography(images: list):
    ''' Computes the pairwise homography from the center image
    
        Args:
            - images: list of N images
            
        Returns:
            - list of warped images to the center image plane
            - list of homographies from the center image
            
    
    '''
    
    # get the center image
    center_img = images[len(images)//2]
    
    # compute pairwise homographies
    homographies = []
    for img in images:
        H = compute_img_homography(img, center_img, num_samples=10, inlier_thresh = 1.5, num_trials=6000) 
        homographies.append(H)
        
    # for
    
    warped_images = []
    for H, img in zip(homographies, images):
        warped_img = inverse_warp(H, img, cols=center_img.shape[1], rows=center_img.shape[0])
        warped_images.append(warped_img)
        
    # for

    mosaicked_img = warped_images[len(images)//2]
    for warped_img in warped_images:
        mosaicked_img = blend_images(warped_img, mosaicked_img)
    
    return mosaicked_img, warped_images, homographies

# pw_homography   



### **Short Answers**

#### 1)

#### 2)

#### 3)

#### 4)

#### 5)